In [ ]:
import json
import pandas as pd
import numpy as np
from shapely.geometry import shape
from pyproj import Transformer

# -------- PATHS --------
GEOJSON_PATH = "data/raw/lct.geojson"
POP_PATH = "data/raw/pop.csv"
OUT_PATH = "data/model_table.csv"

# -------- UKPN GEOJSON PROPERTY KEYS --------
P_LSOA_CODE = "lsoa21cd"
P_LSOA_NAME = "lsoa11nm"
P_TYPE = "type"
P_CONN = "lct_connections"
SOLAR_TYPE = "Solar PV"

# Rough UK bounds (used for sanity + CRS sniff)
UK_LAT = (48, 61)
UK_LON = (-9, 3)

# BNG -> WGS84
bng_to_wgs84 = Transformer.from_crs("EPSG:27700", "EPSG:4326", always_xy=True)

# -----------------------
# LOAD + FLATTEN GEOJSON
# -----------------------
with open(GEOJSON_PATH, "r", encoding="utf-8") as f:
    gj = json.load(f)

rows = []
bad_geom = 0

for feat in gj.get("features", []):
    props = feat.get("properties") or {}
    geom = feat.get("geometry")

    if not geom:
        bad_geom += 1
        continue

    c = shape(geom).centroid
    x, y = float(c.x), float(c.y)

    # If centroid coords already look like UK lon/lat, keep them
    # (y=lat, x=lon). Otherwise assume BNG and transform.
    if UK_LAT[0] <= y <= UK_LAT[1] and UK_LON[0] <= x <= UK_LON[1]:
        lat, lon = y, x
    else:
        lon, lat = bng_to_wgs84.transform(x, y)

    rows.append(
        {
            "lsoa_code": str(props.get(P_LSOA_CODE, "")).strip(),
            "lsoa_name": str(props.get(P_LSOA_NAME, "")).strip(),
            "type": str(props.get(P_TYPE, "")).strip(),
            "connections": float(props.get(P_CONN, 0) or 0.0),
            "latitude": lat,
            "longitude": lon,
        }
    )

df = pd.DataFrame(rows)

# -----------------------
# CLEAN + FILTER SOLAR
# -----------------------
df = df[df["lsoa_code"].ne("")].copy()
df[["latitude", "longitude"]] = df[["latitude", "longitude"]].apply(pd.to_numeric, errors="coerce")
df = df.dropna(subset=["latitude", "longitude"]).copy()

solar = df[df["type"].str.lower().eq(SOLAR_TYPE.lower())].copy()

# Aggregate by LSOA
out = (
    solar.groupby(["lsoa_code", "lsoa_name"], as_index=False)
    .agg(
        solar_connections=("connections", "sum"),
        latitude=("latitude", "mean"),
        longitude=("longitude", "mean"),
    )
)

# -----------------------
# MERGE POPULATION
# -----------------------
pop = pd.read_csv(POP_PATH, usecols=["code", "pop"]).rename(columns={"code": "lsoa_code", "pop": "population"})
pop["lsoa_code"] = pop["lsoa_code"].astype(str).str.strip()
pop["population"] = pd.to_numeric(pop["population"], errors="coerce")

out = out.merge(pop, on="lsoa_code", how="left")
out = out.dropna(subset=["population"]).query("population > 0").copy()

# -----------------------
# METRICS + CATEGORIES
# -----------------------
out["solar_per_1000_pop"] = (out["solar_connections"] / out["population"]) * 1000.0

lat_min, lat_max = out["latitude"].min(), out["latitude"].max()
out["potential_lat_score"] = (lat_max - out["latitude"]) / (lat_max - lat_min + 1e-9)

pot_med = out["potential_lat_score"].median()
up_med = out["solar_per_1000_pop"].median()

out["category"] = np.select(
    [
        (out["potential_lat_score"] >= pot_med) & (out["solar_per_1000_pop"] < up_med),
        (out["potential_lat_score"] >= pot_med) & (out["solar_per_1000_pop"] >= up_med),
        (out["potential_lat_score"] < pot_med) & (out["solar_per_1000_pop"] >= up_med),
    ],
    [
        "High potential / Low uptake (PRIORITY)",
        "High potential / High uptake",
        "Low potential / High uptake",
    ],
    default="Low potential / Low uptake",
)

# Priority: high potential is good; high uptake lowers priority because already adopted
pot = out["potential_lat_score"]
up = out["solar_per_1000_pop"]
out["priority_score"] = (pot - pot.mean()) / (pot.std() + 1e-9) - (up - up.mean()) / (up.std() + 1e-9)

# -----------------------
# FINAL CHECK + SAVE
# -----------------------
out = out[out["latitude"].between(*UK_LAT) & out["longitude"].between(*UK_LON)].copy()
out.to_csv(OUT_PATH, index=False)

print("DONE ✅")
print(f"Wrote: {OUT_PATH}")
print(f"Rows: {len(out):,}")
print(f"Bad geometries: {bad_geom}")
print("Lat range:", float(out["latitude"].min()), "to", float(out["latitude"].max()))
print("Lon range:", float(out["longitude"].min()), "to", float(out["longitude"].max()))

DONE ✅
Wrote: data/model_table.csv
Rows: 10,365
Bad geometries: 0
Lat range: 50.76520195542135 to 52.9553775352635
Lon range: -1.0907394747720862 to 1.7574267991393517
